In [ ]:
import pandas as pd
import json
import random

In [ ]:
system_prompt = """You are a text comprehension expert capable of detecting text generation, distinguishing whether it is authored by a real human or generated by LLMs (Large Language Models).\
    When the user inputs text with "Text DAIGT Detection:" or directly requests you to perform DAIGT detection, you need to determine if the input text is generated by LLMs (Large Language Models) or authored by a human.\
        Additionally, output a floating-point number with two decimal places within the range of [0, 1]. The larger the value, the higher the likelihood that the text is generated by LLMs (Large Language Models); conversely, the higher the likelihood that the text is authored by a human."""

answer_patterns = ['Text DAIGT Detection: ###',
                   'Perform DAIGT detection on the following text: ###',
                   '###. Perform DAIGT detection on the above text',
                   'Whether the following text is generated by LLMs: ###',
                   'Whether the following text is written by a human: ###',
                   'Whether the following text is generated by Large Language Models:###',
                   '###. Is the above text generated by LLMs?',
                   '###. Is the above text generated by Large Language Models?',
                   '###. Is the above text written by a human?']

conversations = []

In [ ]:
# Kaggle上的DAIGT数据基本为csv文件，如下所示，实际使用根据自己的路径修改
data_v3_1 = pd.read_csv("~/train_v3_drcat_01.csv")
data_v3_2 = pd.read_csv("~/train_v3_drcat_02.csv")
data_v4_1 = pd.read_csv("~/daigt_magic_generations.csv")
data_official = pd.read_csv("~/train_essays.csv")

# 删除掉所需字段中为nan的数据行
data_v3_1 = data_v3_1.dropna(subset=['text', 'label'])
data_v3_2 = data_v3_2.dropna(subset=['text', 'label'])
data_v4_1 = data_v4_1.dropna(subset=['text', 'label'])
data_official = data_official.dropna(subset=['text', 'generated'])

In [ ]:
def construct_conversions(data, label='label'):
    random.seed(42)
    for i in range(len(data)):
        text = data['text'].iloc[i]
        flag = data[label].iloc[i]
        patterns_idx = random.randint(0, 8)
        user = answer_patterns[patterns_idx].replace('###', text)
        if flag == 1:
            bot = f'The predicted text to be detected is generated by LLMs (large language models), and the possibility of being generated by LLMs is {flag}'
        else:
            bot = f'The predicted text to be detected is written by a human, and the possibility of being generated by LLMs is {flag}'
        conversation = {
            "system": system_prompt,
            "input": user,
            "output": bot
        }
        conversations_v1.append({"conversation": [conversation]})

In [ ]:
# 因为数据中字段名不一致，未将所有dataframe对象concat一起，而是分开构建
construct_conversions(data_v3_1)
construct_conversions(data_v3_2)
construct_conversions(data_v4_1)
construct_conversions(data_official, label='generated')

In [ ]:
conversations_path = "~/conversations.json"
with open(conversations_v1_path, 'w', encoding='utf-8') as json_file:
    json.dump(conversations_v1, json_file, indent=4)
json_file.close()